In [181]:
# import table
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import json
from opyenxes.data_in.XUniversalParser import XUniversalParser

from pm4py.objects.log.importer.xes import importer
from pm4py.algo.discovery.inductive.variants.im_f import algorithm as im_f_algo
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator

from dpv.util.converter import downscale
from dpv.variant_transformer import public_transform
from dpv.data_utility import emd_utility, log_diff

from privatize_ba import privatize_ba

In [166]:
# settings
event_log = 'sepsis.xes'
cutoff = 19                 # 80% of traces

epsilon = [2, 1, 0.1, 0.01, 0.001]
delta = [0.5, 0.1, 0.05, 0.01, 0.001]
prune_simple = [[3, 3, 3, 3, 3], [3, 3, 3, 3, 3], [3, 8, 10, 13, 14], [4, 13, 22, 70, 110], [4, 15, 28, 108, 600]]

In [3]:
# helper functions

# log reader
def readLogFile(file):
    with open(file) as log_file:
        return XUniversalParser().parse(log_file)

In [8]:
# import xes event log (sepsis example)
log_orig = importer.apply(event_log)
log = readLogFile(event_log)

parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 1201.74it/s]


Unknown extension: http://www.xes-standard.org/meta_org.xesext
Unknown extension: http://www.xes-standard.org/meta_time.xesext
Unknown extension: http://www.xes-standard.org/meta_3TU.xesext
Unknown extension: http://www.xes-standard.org/meta_life.xesext
Unknown extension: http://www.xes-standard.org/meta_concept.xesext
Unknown extension: http://www.xes-standard.org/meta_general.xesext


In [182]:
# process model from xes log
net1, im1, fm1 = im_f_algo.apply(log_orig, parameters=None)

In [168]:
# run SaCoFa with their simple (non-accounted) privacy budgets
for i, eps in enumerate(epsilon):
    for j, d in enumerate(delta):

        # debug output
        print(f"[+] eps: {eps} delta: {d}")

        for k in range(10):

            # privatize variant frequencies
            l = 0
            while l < 100 or l > 500:
                sanitized_frequencies = privatize_ba.privatize_tracevariants(log=log, epsilon=eps, delta=d, P=prune_simple[i][j], N=cutoff)
                sanitized_frequencies = {key.replace('>>>TRACE_END','').replace('>>>',','):val for key,val in sanitized_frequencies.items() if len(key.split('>>>'))>2 and val>0}
                l = len(sanitized_frequencies)

            # save results
            fp = open(f"sacofa/variants_sacofa_{eps}_{d}_{k}.json", "w")
            json.dump(sanitized_frequencies, fp)
            fp.close()


[+] eps: 2 delta: 0.5
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 %
Retrieving true prefix frequencies
Sanitizing...
Retrieving behavioral appropriateness relations
Always follows: 0.0%; Never follows:  36.328125 

In [171]:
# init arrays
eps_list = [2, 1, 0.1, 0.01, 0.001]
del_list = [0.5, 0.1, 0.05, 0.01, 0.001]
util_bench = np.zeros((len(eps_list), len(del_list)))
cost_bench = np.zeros((len(eps_list), len(del_list)))

# compute data utility metrics
for m, eps in enumerate(eps_list):
    for n, d in enumerate(del_list):
        
        tmp_util = []
        tmp_cost = []
        for i in range(10):
        
            f = open(f'sacofa/variants_sacofa_{eps}_{d}_{i}.json',)
            priv_log = json.load(f)
            tmp_util.append(emd_utility(log_orig, priv_log))
            tmp_cost.append(log_diff(log_orig, priv_log))
            f.close()
        
        util_bench[m][n] = np.mean([x for x in tmp_util if x != 1])
        cost_bench[m][n] = np.mean([x for x in tmp_cost if x != 10500])
        print(f'eps: {eps}, delta: {d}, util: {util_bench[m][n]}, cost: {cost_bench[m][n]}')
        
# save metrics
np.savetxt('sacofa/util_sacofa.txt', util_bench)
np.savetxt('sacofa/cost_sacofa.txt', cost_bench)

eps: 2, delta: 0.5, util: 0.6775895356674835, cost: 140500.8
eps: 2, delta: 0.1, util: 0.6920496296633826, cost: 134915.4
eps: 2, delta: 0.05, util: 0.6870480269339809, cost: 136293.7
eps: 2, delta: 0.01, util: 0.6949123161267805, cost: 134052.4
eps: 2, delta: 0.001, util: 0.6900560650691837, cost: 135101.5
eps: 1, delta: 0.5, util: 0.7061147718321809, cost: 128395.1
eps: 1, delta: 0.1, util: 0.7518712650310829, cost: 99154.6
eps: 1, delta: 0.05, util: 0.7553282482585819, cost: 96033.6
eps: 1, delta: 0.01, util: 0.7658195947140817, cost: 90215.9
eps: 1, delta: 0.001, util: 0.7699234866779834, cost: 90362.6
eps: 0.1, delta: 0.5, util: 0.8306209740124846, cost: 35413.0
eps: 0.1, delta: 0.1, util: 0.7717182834709824, cost: 215551.8
eps: 0.1, delta: 0.05, util: 0.753273182884782, cost: 349950.4
eps: 0.1, delta: 0.01, util: 0.7586410634752822, cost: 593927.5
eps: 0.1, delta: 0.001, util: 0.7254604343642821, cost: 435971.9
eps: 0.01, delta: 0.5, util: 0.7287002901476842, cost: 90305.9
eps: 0

In [183]:
# init arrays
eps_list = [2, 1, 0.1, 0.01, 0.001]
del_list = [0.5, 0.1, 0.05, 0.01, 0.001]
prec_bench = np.zeros((len(eps_list), len(del_list)))
fit_bench = np.zeros((len(eps_list), len(del_list)))
gen_bench = np.zeros((len(eps_list), len(del_list)))

for m, eps in enumerate(eps_list):
    for n, d in enumerate(del_list):
        
        prec_bench_avg = []
        fit_bench_avg = []
        gen_bench_avg = []

        for i in range(10):
            
            f = open(f'sacofa/variants_sacofa_{eps}_{d}_{i}.json',)
            priv_log = json.load(f)
            
            if len(priv_log) == 0:
                f.close()
                continue
            
            # compute result metrics
            scaled_priv = downscale(priv_log, type='all')
            plog = public_transform(scaled_priv)
            prec = precision_evaluator.apply(plog, net1, im1, fm1, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
            fit = replay_fitness_evaluator.apply(plog, net1, im1, fm1, variant=replay_fitness_evaluator.Variants.TOKEN_BASED)
            gen = generalization_evaluator.apply(plog, net1, im1, fm1)
            
            prec_bench_avg.append(prec)
            fit_bench_avg.append(fit['log_fitness'])
            gen_bench_avg.append(gen)
            
            f.close()
        
        # save results
        prec_bench[m][n] = np.mean(prec_bench_avg)
        fit_bench[m][n] = np.mean(fit_bench_avg)
        gen_bench[m][n] = np.mean(gen_bench_avg)
        print(f'eps: {eps}, delta: {d}, prec: {prec_bench[m][n]}, fit: {fit_bench[m][n]}, gen: {gen_bench[m][n]}')

# save metrics
np.savetxt('sacofa/prec_sacofa.txt', prec_bench)
np.savetxt('sacofa/fit_sacofa.txt', fit_bench)
np.savetxt('sacofa/gen_sacofa.txt', gen_bench)

replaying log with TBR, completed variants :: 100%|██████████| 132/132 [00:00<00:00, 687.52it/s]


eps: 2, delta: 0.5, prec: 0.4405092320591926, fit: 0.8805394701578203, gen: 0.8324412176265257


replaying log with TBR, completed variants :: 100%|██████████| 155/155 [00:00<00:00, 674.09it/s]


eps: 2, delta: 0.1, prec: 0.45234799742135745, fit: 0.8845558352084989, gen: 0.845185537557209


replaying log with TBR, completed variants :: 100%|██████████| 137/137 [00:00<00:00, 769.91it/s]


...

eps: 0.001, delta: 0.05, prec: 0.36467394987112545, fit: 0.8600619776761746, gen: 0.9617332090146491


replaying log with TBR, completed variants :: 100%|██████████| 229/229 [00:00<00:00, 489.80it/s]


eps: 0.001, delta: 0.01, prec: 0.317614337563137, fit: 0.8134766392115559, gen: 0.9680051429717096


replaying log with TBR, completed variants :: 100%|██████████| 366/366 [00:01<00:00, 247.14it/s]

eps: 0.001, delta: 0.001, prec: 0.3345421966253367, fit: 0.8121594919010839, gen: 0.9559794311421047
